# How to handle "double-texting" or concurrent runs in your graph

You might want to start a new run on a thread while the previous run still haven't finished. We call this "double-texting" or multi-tasking.

There are several strategies for handling this:
 
- `reject`: Reject the new run
- `cancel`: Cancel the existing run and start a new one.
- `rollback`: Cancel and delete the existing run, rolling back thread to the state before it had started.
- `enqueue`: Wait until the existing run is finished to start the new run.

### Reject

In [1]:
from langgraph_sdk import get_client
from langchain_core.messages import convert_to_messages
import httpx

In [2]:
client = get_client()

In [3]:
assistant = await client.assistants.create("agent")

In [4]:
thread = await client.threads.create()

In [5]:
run = await client.runs.create(
    thread["thread_id"],
    assistant["assistant_id"],
    input={"messages": [{"role": "human", "content": "whats the weather in sf?"}]}
)

In [6]:
try:
    await client.runs.create(
        thread["thread_id"],
        assistant["assistant_id"],
        input={"messages": [{"role": "human", "content": "whats the weather in nyc?"}]},
        multitask_strategy="reject",
    )
except httpx.HTTPStatusError as e:
    print("Failed to start concurrent run", e)

Failed to start concurrent run Client error '409 Conflict' for url 'http://localhost:8123/threads/c0d9cf34-934c-4d01-813e-7828f2e725f0/runs'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/409


We can verify that the original thread finished executing:

In [7]:
state = await client.threads.get_state(thread["thread_id"])

In [8]:
for m in convert_to_messages(state["values"]["messages"]):
    m.pretty_print()

================================ Human Message =================================

whats the weather in sf?
================================== Ai Message ==================================

[{'id': 'toolu_019mUvmKJHarkWEi4SxfX717', 'input': {'query': 'weather in san francisco'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}]
Tool Calls:
  tavily_search_results_json (toolu_019mUvmKJHarkWEi4SxfX717)
 Call ID: toolu_019mUvmKJHarkWEi4SxfX717
  Args:
    query: weather in san francisco
================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1718242421, 'localtime': '2024-06-12 18:33'}, 'current': {'last_updated_epoch': 1718242200, 'last_updated': '2024-06-12 18:30', 'temp_c': 20.6, 'temp_f':

### Cancel

In [9]:
thread = await client.threads.create()

In [10]:
# this run will be canceled
canceled_run = await client.runs.create(
    thread["thread_id"], assistant["assistant_id"],
    input={"messages": [{"role": "human", "content": "whats the weather in sf?"}]}
)

In [11]:
run = await client.runs.create(
    thread["thread_id"],
    assistant["assistant_id"],
    input={"messages": [{"role": "human", "content": "whats the weather in nyc?"}]},
    multitask_strategy="cancel",
)

We can see that the thread only has data from the second run

In [ ]:
# wait until the second run completes
await client.runs.join(thread["thread_id"], run["run_id"])

In [19]:
state = await client.threads.get_state(thread["thread_id"])

In [20]:
# TODO: after fixes this should be < 8
for m in convert_to_messages(state["values"]["messages"]):
    m.pretty_print()

================================ Human Message =================================

whats the weather in sf?
================================== Ai Message ==================================

[{'id': 'toolu_01GuYpX4P6H9E62Ai4GX7i12', 'input': {'query': 'weather in san francisco'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}]
Tool Calls:
  tavily_search_results_json (toolu_01GuYpX4P6H9E62Ai4GX7i12)
 Call ID: toolu_01GuYpX4P6H9E62Ai4GX7i12
  Args:
    query: weather in san francisco
================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1718242617, 'localtime': '2024-06-12 18:36'}, 'current': {'last_updated_epoch': 1718242200, 'last_updated': '2024-06-12 18:30', 'temp_c': 20.4, 'temp_f':

Verify that the original, canceled run was interrupted

In [21]:
(await client.runs.get(thread["thread_id"], canceled_run["run_id"]))["status"]

'interrupted'

### Rollback

In [25]:
thread = await client.threads.create()

In [26]:
# this run will be canceled
rolled_back_run = await client.runs.create(
    thread["thread_id"],
    assistant["assistant_id"],
    input={"messages": [{"role": "human", "content": "whats the weather in sf?"}]}
)

In [27]:
run = await client.runs.create(
    thread["thread_id"],
    assistant["assistant_id"],
    input={"messages": [{"role": "human", "content": "whats the weather in nyc?"}]},
    multitask_strategy="rollback",
)

We can see that the thread only has data from the second run

In [28]:
# wait until the second run completes
await client.runs.join(thread["thread_id"], run["run_id"])

In [29]:
state = await client.threads.get_state(thread["thread_id"])

In [30]:
# TODO: after fixes this should be < 8 messages
for m in convert_to_messages(state["values"]["messages"]):
    m.pretty_print()

================================ Human Message =================================

whats the weather in sf?
================================== Ai Message ==================================

[{'id': 'toolu_018XxkyjJQY9SwsjsJGwRxCQ', 'input': {'query': 'weather in san francisco'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}]
Tool Calls:
  tavily_search_results_json (toolu_018XxkyjJQY9SwsjsJGwRxCQ)
 Call ID: toolu_018XxkyjJQY9SwsjsJGwRxCQ
  Args:
    query: weather in san francisco
================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1718242617, 'localtime': '2024-06-12 18:36'}, 'current': {'last_updated_epoch': 1718242200, 'last_updated': '2024-06-12 18:30', 'temp_c': 20.4, 'temp_f':

Verify that the original, rolled back run was deleted -- we should get the error message here

In [31]:
try:
    await client.runs.get(thread["thread_id"], rolled_back_run["run_id"])
except httpx.HTTPStatusError as e:
    print("Couldn't retrieve deleted run", e)

Couldn't retrieve deleted run Client error '404 Not Found' for url 'http://localhost:8123/threads/54254cc2-b487-4346-a079-6546af258079/runs/1ef2925b-198f-6fc6-810e-b991b59e4352'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/404


### Enqueue

In [32]:
thread = await client.threads.create()

In [33]:
# this run will be canceled
first_run = await client.runs.create(
    thread["thread_id"],
    assistant["assistant_id"],
    input={"messages": [{"role": "human", "content": "whats the weather in sf?"}]}
)

In [34]:
second_run = await client.runs.create(
    thread["thread_id"],
    assistant["assistant_id"],
    input={"messages": [{"role": "human", "content": "whats the weather in nyc?"}]},
    multitask_strategy="enqueue",
)

Verify that the thread has data from both runs

In [ ]:
# wait until the second run completes
await client.runs.join(thread["thread_id"], second_run["run_id"])

In [ ]:
state = await client.threads.get_state(thread["thread_id"])

In [ ]:
for m in convert_to_messages(state["values"]["messages"]):
    m.pretty_print()